# Baseline model for recruiting data

In this notebook we train a simple model on the recruiting data that can serve as a counterfactual for what would have happened if we hadn't made any kind of fairness intervention.

In [ ]:
import joblib
from pathlib import Path

import numpy as np
import pandas as pd
from helpers.recruiting import bin_years_experience
from helpers.plot import group_box_plots
from sklearn.neural_network import MLPClassifier  # noqa

Directory containing preprocessed data.

In [ ]:
artifacts_dir = Path("../../artifacts")

Load the preprocessed data. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

# used for unscaled years of experience
test_raw = pd.read_csv(data_dir / "raw" / "test.csv")

## Training a model to predict salary

We will load a model from disk so that results are reproducible, but commented out here is the code we used to train the model.

In [ ]:
# model = MLPClassifier(hidden_layer_sizes=(100, 100), early_stopping=True)

# model.fit(train.drop(columns="employed_yes"), train.employed_yes)

Load the pretrained model

In [ ]:
model = joblib.load(artifacts_dir / "models" / "recruiting" / "baseline.pkl")

Model accuracy on validation set

In [ ]:
test_prob = model.predict_proba(test.drop(columns="employed_yes"))[:, 1]
test_accuracy = model.score(
    test.drop(columns="employed_yes"), test.employed_yes
)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

## Demographic parity

Distribution of outcomes for different sexes.

In [ ]:
fig_dp_by_sex = group_box_plots(
    test_prob,
    np.zeros_like(test_prob),
    test.sex_male.map(lambda x: "Male" if x else "Female"),
    group_names=[""],
)
fig_dp_by_sex

Distribution of outcomes for different races.

In [ ]:
fig_dp_by_race = group_box_plots(
    test_prob,
    np.zeros_like(test_prob),
    test.race_white.map(lambda x: "White" if x else "Black"),
    group_names=[""],
)
fig_dp_by_race

## Conditional demographic parity

Distribution by sex and hours worked per week.

In [ ]:
test_exp_enum = test_raw.years_experience.map(bin_years_experience)

fig_cdp_by_sex = group_box_plots(
    test_prob,
    test_exp_enum,
    test.sex_male.map(lambda x: "Male" if x else "Female"),
    group_names=["0-2 years", "3-5 years", "6-9 years", "10+ years"],
)
fig_cdp_by_sex

Distribution by race and hours worked per week.

In [ ]:
fig_cdp_by_race = group_box_plots(
    test_prob,
    test_exp_enum,
    test.race_white.map(lambda x: "White" if x else "Black"),
    group_names=["0-2 years", "3-5 years", "6-9 years", "10+ years"],
)
fig_cdp_by_race

## Equalised odds

To assess equalised odds we compare scores across the outcome classes.

In [ ]:
fig_eo_by_sex = group_box_plots(
    test_prob,
    test.employed_yes,
    test.sex_male.map(lambda x: "Male" if x else "Female"),
    group_names=["Not employed", "Employed"],
)
fig_eo_by_sex

We do the same, comparing races.

In [ ]:
fig_eo_by_race = group_box_plots(
    test_prob,
    test.employed_yes,
    test.race_white.map(lambda x: "White" if x else "Black"),
    group_names=["Not employed", "Employed"],
)
fig_eo_by_race